In [1]:
import meshio as mio
import numpy as np
from scipy.io import savemat
import meshplot as mp
import torch
import numpy as np
import matplotlib.pyplot as plt
import igl

In [2]:
def vec_mass(rho, V, T):
    Vs = V[T, :]
    vol = torch.linalg.det(Vs[:, 1:4, :] - Vs[:, 0:3, :])
    tmp = rho*vol
    
    mass = torch.sum(tmp)
    
    return mass

In [3]:
def vec_cm(rho, V, T):
    Vs = V[T, :]
    vol = torch.linalg.det(Vs[:, 1:4, :] - Vs[:, 0:3, :])
    tmp = rho*vol
    
    cm = torch.sum(tmp*torch.mean(Vs, axis=1).T, axis=1)
    mass = torch.sum(tmp)
    
    return cm / mass

In [4]:
def vec_energy(r, newcm, V, T):
    return torch.sum((newcm - vec_cm(r,V,T))**2);

In [5]:
m = mio.read("gargoyle.msh")
V = m.points
T = m.cells[0].data
input_rho = 1.11


V = torch.tensor(V, dtype = torch.float64)
T = torch.tensor(T, dtype = torch.int64)

rt = torch.ones(T.shape[0], dtype = torch.float64)*input_rho
target = vec_cm(rt, V, T).detach()
target_mass = vec_mass(rt, V, T).detach()
print(target.numpy(), target_mass.numpy())


####################################

m = mio.read("gargoyle-coarse.msh")
V = m.points
T = m.cells[0].data

V = torch.tensor(V, dtype = torch.float64)
T = torch.tensor(T, dtype = torch.int64)


[-4.99457255 -0.35278529  1.86683635] 123.97748336298699



In [11]:
# L-BFGS
r0 = torch.ones(T.shape[0], requires_grad=True, dtype = torch.float64)
# target = torch.tensor([0.4,0,0])
print(vec_cm(r0, V, T).detach().numpy())

optimizer = torch.optim.LBFGS([r0],
                        history_size=10,
                        max_iter=4,
                        line_search_fn="strong_wolfe")


f = lambda r: vec_energy(r, target, V, T)
vv = []
for i in range(1000):
    if i % 50 == 0:
        print(i)
    optimizer.zero_grad()
    objective = f(r0)
    objective.backward()
    optimizer.step(lambda: f(r0))
    vv.append(objective.item())
    
    if objective.item() < 1e-10:
        break
    
print(vv[-1])
# plt.semilogy(vv)

opt = r0.detach()
opt_mass = vec_mass(opt, V, T).detach()
opt *=  target_mass/opt_mass
opt_mass = vec_mass(opt, V, T).detach()
opt = opt.numpy()

print("opt:", vec_cm(r0, V, T).detach().numpy(), "vs target:", target.numpy())
print("opt:", opt_mass.numpy(), "vs target:", target_mass.numpy())

print("start:", np.sqrt(vv[0]),"opt:",np.sqrt(vv[-1]))
print("min rho:", np.min(opt), "max rho:",np.max(opt))

[-4.99307055 -0.35527529  1.86930643]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1.1040282970515243e-10
opt: [-4.9945784  -0.35278117  1.86682866] vs target: [-4.99457255 -0.35278529  1.86683635]
opt: 123.97748336298699 vs target: 123.97748336298699
start: 0.00381541423630451 opt: 1.050727508468073e-05
min rho: 1.1002821223025225 max rho: 1.1457655456703004


In [12]:
np.savetxt("rhos.txt", opt*1000)
!head rhos.txt

1.123617475864233484e+03
1.124838774734135541e+03
1.125122275581669101e+03
1.122988821744089591e+03
1.121117169549749406e+03
1.123153010924729415e+03
1.124248368072963785e+03
1.123044265833254258e+03
1.118778588644108595e+03
1.123073333187630624e+03
